# Waymo Dataset 

To Be Added:
Project Overview
Goals:
Data Type:


In [ ]:
!pip3 install waymo-open-dataset
import os
import tensorflow as tf
import math
import numpy as np
import itertools

tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

Lets Look at One of the files and what it contains

In [ ]:
filename = "segment-10206293520369375008_2796_800_2816_800_with_camera_labels.tfrecord"
dataset=tf.data.TFRecordDataset(filename,compression_type='')
for data in dataset:
  frame=open_dataset.Frame()
  frame.ParseFromString(bytearray(data.numpy()))
  #Get the location
  print("The location is %s "%frame.context)
  #Get the number of cars,signs and pedestrians
  
  cars=frame.context.stats.laser_object_counts[0].count
  signs=frame.context.stats.laser_object_counts[1].count
  print("Number of Cars: %s,\nNumber of signs: %s"%(cars,signs))
  break
'''
stats {
laser_object_counts {
type: TYPE_VEHICLE
count: 6
}
laser_object_counts {
type: TYPE_SIGN
count: 4
}
time_of_day: "Night"
location: "location_phx"
weather: "sunny"
camera_object_counts {
type: TYPE_VEHICLE
count: 4
}
}
'''


Lets Take a look at the image being used
This is taken from the colab tutorial (LINK TO)


In [ ]:
#Used for showing the pictures
import matplotlib.pyplot as plt
#Used for creating the bouding boxes
import matplotlib.patches as patches

def show_camera_image(camera_image, camera_labels, layout, cmap=None):
  """Show a camera image and the given camera labels."""
  #Create a general area for the output of the function
  ax = plt.subplot(*layout)

  # Draw the camera labels.
  for camera_labels in frame.camera_labels:
    # Ignore camera labels that do not correspond to this camera.
    if camera_labels.name != camera_image.name:
      continue

    # Iterate over the individual labels.
    for label in camera_labels.labels:
      # Draw the object bounding box.
      ax.add_patch(patches.Rectangle(
        xy=(label.box.center_x - 0.5 * label.box.length,
            label.box.center_y - 0.5 * label.box.width),
        width=label.box.length,
        height=label.box.width,
        linewidth=1,
        edgecolor='red',
        facecolor='none'))

  # Show the camera image.
  plt.imshow(tf.image.decode_jpeg(camera_image.image), cmap=cmap)
  plt.title(open_dataset.CameraName.Name.Name(camera_image.name))
  plt.grid(False)
  plt.axis('off')

plt.figure(figsize=(25, 20))

for index, image in enumerate(frame.images):
  show_camera_image(image, frame.camera_labels, [3, 3, index+1])

In [ ]:
def weather_totals():
    '''
    Gets the weather amount for both
    '''
def augment_weather_images:

In [ ]:
from PIL import Image
from tqdm import tqdm
import os
import pickle
direc='/data/Shared drives/Waymo Project/Data'
#Need to Create a List of Dictionaries in the following format, will make it a pickle
'''
roidbs (list[dict]):
        Produce "roidbs" as a list of dict, each dict corresponds to one image with k>=0 instances.
        and the following keys are expected for training:
        file_name: str, full path to the image
        boxes: numpy array of kx4 floats, each row is [x1, y1, x2, y2]
        class: numpy array of k integers, in the range of [1, #categories], NOT [0, #categories)
        is_crowd: k booleans. Use k False if you don't know what it means.
        
boundingboxtypes=["unknown","Vehicle"]
boundingboxlist=[]
dataset=tf.data.TFRecordDataset(file,compression_type='')
for data in dataset:
      frame=open_dataset.Frame()
      frame.ParseFromString(bytearray(data.numpy()))
      break
for camlabel in frame.camera_labels:
  for label in camlabel.labels:

    boundingboxlist.append([file,
                            label.box.center_x - 0.5 * label.box.length,
                            label.box.center_y - 0.5 * label.box.width,
                            label.box.center_x + 0.5 * label.box.length,
                            label.box.center_y + 0.5 * label.box.width,
                            boundingboxtypes[label.type]])
#for index, image in enumerate(frame.images):
# show_camera_image(image, frame.camera_labels, [3, 3, index+1])

print(boundingboxlist)
'''

# Use this funciton to save the images for presentation
def saveimagesmaskrcnn(d):
  toBePickled=[]
  labelsMsg = open_dataset.CameraLabels()
  
  switch = False #set to true if you wanna save

# only use one file, change to static file (200 images in a frame sequence and 5 sides)
  with os.scandir(d) as it:
    for entry in tqdm(it):
      if entry.name.endswith('.tfrecord'):
        imageLabels={}
        filename=entry.path
        # main line
        dataset=tf.data.TFRecordDataset(filename)
        
        d_name= entry.name.split("-")[-1][:-9]
# use frame images at a certain place to find certain image
# do imagename.image to get actual image
# 
        i_frame = 0
        # go through each of 200 images and operate on eachone
        for data in dataset:
          #exact these 2
          frame=open_dataset.Frame()
          frame.ParseFromString(bytearray(data.numpy()))
          #Save image to new folder and use the name in dict
          counter = 0
          for lbl,img in zip(frame.camera_labels,frame.images):
            name=f"{d_name}_frame_{i_frame:05}_image_{counter:05}.png"

            if switch:
              # use these 3 lines to go from array of stuff to an actual image
              #im.show shows video of the image
              #show_camera_image function above to get from a file to all the images
              # can add a rectangle with the actual box, save and GIF
              decoded=tf.image.decode_jpeg(img.image)
              tonumpy=decoded.numpy()
              im=Image.fromarray(tonumpy)
              im.save(os.path.join('/data/Shared drives/Waymo Project/Data/FastImages',name))           

            labelsMsg.CopyFrom(lbl)
            boxes = np.array([[label.box.center_x - 0.5 * label.box.length,
                              label.box.center_y - 0.5 * label.box.width,
                              label.box.center_x + 0.5 * label.box.length,
                              label.box.center_y + 0.5 * label.box.width] for label in lbl.labels])
            labels = np.array([label.type+1 for label in labelsMsg.labels])

            toBePickled.append({'file_name':name,'boxes':boxes, 'class':labels, 'is_crowd':False})
            counter+=1
          i_frame += 1

  return toBePickled
              
import pickle
p=saveimagesmaskrcnn(direc)
with open(os.path.join('/data/Shared drives/Waymo Project/Data/FastImages',"via_region_data.pickle.pickle"),'wb') as fileObj:
  pickle.dump(p,fileObj)


with open(os.path.join('/data/Shared drives/Waymo Project/Data/FastImages',"via_region_data.pickle.pickle"),'rb') as istream:
    my_pickle = pickle.load(istream)
    # obj = [{"file_name":e['file'],'boxes':e['boxes'],'class':e['labels'],'is_crowd':e['is_crowd']} for e in pickle.load(istream)]
    # pickle.dump(obj,ostream)
    #print(obj)

import pandas as pd
df_pickle = pd.DataFrame(my_pickle)
df_pickle.describe()

In [ ]:
#Preprocessing Pipeline! Returns a list of images and their respective labels

def preprocess(directory):
  #The eventual list of all images
  
  images=[]
  #The eventual list of all labels where (0,1) is rainy and (1,0) is sunny
  labels=[]
  for root,dirs,files in os.walk(directory):
    for file in files:
      if file.endswith(".png"):
        fullpath=os.path.join(root,file)
        #Convert to a One-Hot-Encoding
        if "sunny" in file: 
          label=((1,0))
        else:
          label=((0,1))
        images.append(fullpath)
        labels.append(label)
  assert len(images)==len(labels)
  return images,labels
direct="YOUR FILEPATH HERE"
preprocess(direct)

In [ ]:
batch_size = 7
num_classes = 2
epochs = 13
num_predictions = 20
DIR = "/data/Shared drives/Waymo Project/Data/Images"

files = list(os.scandir(DIR))
X = [f.name for f in files]
y = ['sun' if f.name.startswith('sun') else 'rain' for f in files]

suns = sum((1 for w in y if w=='sun'))
print(suns,len(y)-suns)  

# The data, split between train and test sets:
name_train,  name_test, y_train, y_test =  train_test_split(X, y, test_size=0.33, random_state=42)

model = Sequential([
  Conv2D(32, (3, 3), padding='same',input_shape=(128,128,3)),
  Activation('relu'),
  Conv2D(32, (3, 3)),
  Activation('relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),

  Conv2D(64, (3, 3), padding='same'),
  Activation('relu'),
  Conv2D(64, (3, 3)),
  Activation('relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),

  Flatten(),
  Dense(512),
  Activation('relu'),
  Dropout(0.5),
  Dense(num_classes),
  Activation('softmax')
])


# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop()

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


#print('Not using data augmentation.')
#model.fit(x_train, y_train,
#          batch_size=batch_size,
#          epochs=epochs,
#          validation_data=(x_test, y_test),
#          shuffle=True)



# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
#datagen.fit(x_train)

srcGen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    DataFrame(data={'filename':name_train,'class':y_train}),directory=DIR,target_size=(128,128),batch_size=16)

dataGen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    DataFrame(data={'filename':name_test,'class':y_test}),
    directory=DIR,target_size=(128,128),batch_size=16,)

# Fit the model on the batches generated by datagen.flow().
checkpointer = ModelCheckpoint(filepath='/data/Shared drives/Waymo Project/tmp/weights.hdf5', verbose=1, save_best_only=True)
model.fit_generator(srcGen,
                    epochs=epochs,
                    validation_data=dataGen,
                    workers=4,
                    callbacks=[checkpointer])


# Score trained model.
scores = model.evaluate(dataGen, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])